# Imaging steps

## Step 1: Input & setup

In [28]:
import subprocess
# import multiprocess
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [29]:
plt.style.use("default")
plt.rc('xtick', direction='in', top=True)
plt.rc('ytick', direction='in', right=True)
plt.rc('axes', linewidth=1.15)

plt.rc("mathtext", fontset="dejavuserif")

In [30]:
if not os.path.exists('../Data/Images'):
    os.system('mkdir ../Data/Images')

event_file = '../Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits'

## Creating Image and exposure map

In [31]:
def run_evtool(input_name, output_name, emin, emax, gti_type='FLAREGTI', flag_type='0xe00fff30', pattern='15', telid='1 2 3 4 5 6 7', log_file=None):
    subprocess.run(['evtool', 
                    f'eventfiles={input_name}', 
                    f'outfile={output_name}', 
                    f'gti={gti_type}', 
                    f'flag={flag_type}', 
                    f'pattern={pattern}', 
                    f'emin={emin}', 
                    f'emax={emax}',
                    f'image=yes',
                    f'events=no',
                    f'telid={telid}'
                    ],
                    stdout=log_file,
                    stderr=log_file)
    
def run_expmap(input_eventlist, input_image, output_name, emin, emax, log_file=None):
    subprocess.run(['expmap', 
                    f'inputdatasets={input_eventlist}', 
                    f'templateimage={input_image}', 
                    f'mergedmaps={output_name}', 
                    f'emin={emin}', 
                    f'emax={emax}',
                    'withvignetting=yes',
                    'withweights=yes',
                    ],
                    stdout=log_file,
                    stderr=log_file)

In [32]:
band_min = 0.2
band_max = 2.3
run_evtool(event_file, f'../Data/Images/merged_image_{int(band_min*1000)}_{int(band_max*1000)}.fits', 
           band_min, band_max, log_file=open(f'../Data/Images/merged_image_{int(band_min*1000)}_{int(band_max*1000)}.log', 'w+'))
run_expmap(event_file, f'../Data/Images/merged_image_{int(band_min*1000)}_{int(band_max*1000)}.fits', 
           f'../Data/Images/merged_expmap_{int(band_min*1000)}_{int(band_max*1000)}.fits', 
           band_min, band_max, log_file=open(f'../Data/Images/merged_expmap_{int(band_min*1000)}_{int(band_max*1000)}.log', 'w+'))